<a href="https://colab.research.google.com/github/patriciacs99/WeaponDetectionYOLOv5/blob/main/WeaponDetectionYOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔫 Firearms detection with YOLOV5

In this project the following code snippets have been used to detect firearms and classify them into long guns (shotguns, rifles, machine guns and snipers) and handguns (pistols and revolvers).

For the experiments performed, the necessary cells will be executed for each of them depending on the objective of the experiment. In each case changing the necessary values as indicated in this notebook.
https://drive.google.com/drive/folders/15O3lpCT-JYyuhEc5WftPzr0vCELjLPSS?usp=drive_link  


---

In [ ]:
#imports
import os
import shutil
import random
import cv2
import torch
import matplotlib.pyplot as plt
import torchvision
import numpy as np
import glob
import re
import time
from pathlib import Path
import itertools
from IPython.display import Image,display
from google.colab.patches import cv2_imshow

In [ ]:
# Check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

##Prepare data and Yolov5



>In this section we will access Google Drive, where we will have organized the datasets to be used in the experiments.   
We will clone the Ultralytics YOLOv5 repository to be able to work with it and install its requirements.  
We will also download the YOLOv5 models with weights pre-trained with the COCO dataset we are going to use. In our case, we will only use YOLOv5s and YOLOv5m.

⚠ *As a result of the execution of the YOLOv5 files (train.py,detect.py and val.py) a "runs" folder will be generated in the YOLOv5 directory where we will find the results of the executions.*







In [ ]:
#Connecting Google Drive

from google.colab import drive
drive.mount('/content/drive',force_remount=True)


In [ ]:
# Clone YOLOv5
%cd content
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
#Install YOLOv5
%cd /content/yolov5
%pip install -qr requirements.txt

In [ ]:
# Download pre trained models
'''
  Modelos a probar:
    - yolov5s.pt
    - yolov5m.pt
    - yolov5l.pt
    - yolov5x.pt
'''
# Replace "yolov5m.pt" with the preferred model name
!wget https://github.com/ultralytics/yolov5/releases/download/v6.1/yolov5m.pt

## 👟 Model training


> We will train the model for our particular case: detecting short and long firearms.  
In the following cells, they should be replaced by their corresponding routes:
*   **conf_file_path**: path to the yaml file of the dataset used.
*   **weigths_model_YOLOv5_path**: path to the model with the pre-trained weights previously downloaded.
*   **results_path**: path to the directory where the results have been generated.





In [ ]:
!python train.py --img 416 --batch 32 --epochs 100 --data conf_file_path --weights weigths_model_YOLOv5_path   --name "resultados_entrenamiento" --nosave

In [ ]:
#Download training results
from google.colab import files

!zip -r train_results_yolov5.zip results_path/resultados_entrenamiento
files.download('train_results_yolov5.zip')

### Tensorboard Training Results


> We will run Tensorboard to visualize the graphs resulting from the training.  
Replace:
*   **training_path**: path of the directory where the training results have been generated.









In [ ]:
%reload_ext tensorboard


In [ ]:
%tensorboard --logdir training_path


## 🎯 Inference

> The weights of the trained model will be used to detect and classify weapons in new test images that the model has not seen before.  In the following cells, they should be replaced by their corresponding paths:
*   **test_images_path**:path to the test images of the dataset.
*   **weights_path**: path to weights generated in training.
*   **detections_path**: path to the directory where the results of the inference have been generated.
*   **results_path**: path to the directory where the results have been generated.








In [ ]:
!python detect.py --source test_images_path  --weights weights_path --img 416 --save-conf  --save-txt

In [ ]:
#Show test images detections
for imageName in glob.glob("detections_path/*.jpg"):
  display(Image(filename=imageName))
  print("\n")

In [ ]:
#Download results
from google.colab import files

!zip -r train_detect_yolov5.zip results_path
files.download('train_detect_yolov5.zip')

## 📈 Evaluation


>   

> We will extract the metrics of the model on the new test images in order to evaluate its performance.  
In the following cells, they should be replaced by their corresponding paths:
*   **conf_file_path**: path to the yaml file of the dataset used.
*   **weights_path**:path where the file with the weights resulting from the training of the model is located.
*   **results_path**: path to the directory where the results of the execution of the val.py file have been generated.












In [ ]:
!python val.py --weights weights_path --data conf_file_path --workers 0 --img 416 --save-txt --conf-thres 0.5 --task test

In [ ]:
#Download results
from google.colab import files

!zip -r train_metrics_yolov5.zip results_path
files.download('train_metrics_yolov5.zip')



---

## 📑 Additional code


> The following code snippets have been useful for the elaboration of the experiments.



### Intersection Over Union


>Although YOLO calculates the IoU itself, here the intersections are calculated for each image in order to draw them on each image.   
When executing the following cells, the intersection between the real label and the detection is calculated, and the Ground Truth is drawn on the image containing the detection in order to clearly visualize the difference.



In [ ]:
 #Format: x,y,w,h  ---> Format: x1,y1,x2,y2
def rectangulo(box):
    box_x1 = box[0] - box[2]/2
    box_y1 = box[1] - box[3]/2
    box_x2 = box[0] + box[2]/2
    box_y2 = box[1] + box[3]/2
    return [box_x1,box_y1,box_x2,box_y2]

In [ ]:
def intersection_over_union(target_bbox,predicted_bbox):

    #Format: x,y,w,h

    #Calculate corners bboxes
    tg = rectangulo(target_bbox)
    pred = rectangulo(predicted_bbox)

    #Calculate corners interseccion
    x1 = max(pred[0], tg[0])
    y1 = max(pred[1], tg[1])
    x2 = min(pred[2], tg[2])
    y2 = min(pred[3], tg[3])

    #area = w * h
    intersection = max(0,(x2 - x1)) * max(0,(y2 - y1))

    box1_area = abs((pred[2] - pred[0]) * (pred[3] - pred[1]))
    box2_area = abs((tg[2] - tg[0]) * (tg[3] - tg[1]))S

    return intersection / (box1_area + box2_area - intersection + 1e-16)


In [ ]:
#Draw detections,IoU,groundTruth

predicted = "/content/yolov5/runs/detect/exp/labels/0205_jpg.rf.eb76fc2a71b61cec8cdf25fd3e51c6cb.txt"
real = "/content/drive/MyDrive/Deteccion_armas_TFG/Datasets/Customized/WeaponsDataset.v8-weapons_strectch_oclussionsomited.yolov5pytorch/test/images/0205_jpg.rf.eb76fc2a71b61cec8cdf25fd3e51c6cb.jpg"

#Read detection image
image = cv2.imread("/content/drive/MyDrive/Deteccion_armas_TFG/Datasets/weapons.v2-weapons_blackedges_contrast.yolov5pytorch/test/images/0251_jpg.rf.1d75a35dbae43d7e1f13a8266a5d8726.jpg")

with open(real) as archivo: # Open gt coordenates file
    for linea in archivo:
      coord1 =[] # file coordenates storing list
      clase = linea[0] # getting detection class - first character of the read line
      coord1.append(linea[1:].split()) # Removing first character and adding remaining coordenates
      coord1 =list(itertools.chain(*coord1)) # Flattening list
      x=0
      for i in coord1: # Casting values
            coord1[x]= float(i)
            x +=1
      coords= [] # Coordinates of all possible detections will have to be saved for comparison with the gt.
      if os.path.exists(predicted):
        with open(predicted) as archivo:
          for linea in archivo:
                coord2 =[]
                coord2.append(linea[1:].split())
                coord2= list(itertools.chain(*coord2))
                x=0
                for i in coord2:
                        coord2[x]= float(i)
                        x +=1
                coords.append(coord2)

        ious = [] # Storing ious
        for i in coords:
            ious.append(intersection_over_union(i,coord1))
        # Selecting larger IoU value detection
        mayor = ious.index(max(ious))
        deteccion = coords[mayor]

        pred = rectangulo(deteccion)
        gt = rectangulo(coord1)
        iou = max(ious)

      else:
        pred = rectangulo([0,0,0,0])
        gt = rectangulo(coord1)
        iou = 0.000

      for i in range(0,4):
          gt[i] = int(gt[i]*416)

      start_point = tuple(gt[:2])
      end_point = tuple(gt[2:])

      #Print rectangles on detection image
      cv2.rectangle(image,start_point,end_point,(0, 255, 0), 4)

      cv2.putText(image, "IoU: {:.3f}".format(iou), (gt[0]-10, gt[1]-10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
      if iou !=0:
        cv2.putText(image, "Class: "+ clase, (gt[0]+105, gt[1]- 10),
              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0, 0), 2)

# Show image with IoU value, detection and ground truth
cv2_imshow(image)


### Size of image instances


> Code to divide test images according to their size used in experiment 2.



In [ ]:
# Create 6 directories
os.makedirs("dir_peq/images")
os.makedirs("dir_peq/labels")
os.makedirs("dir_med/images")
os.makedirs("dir_med/labels")
os.makedirs("dir_gra/images")
os.makedirs("dir_gra/labels")
os.makedirs("dir_mez/images")
os.makedirs("dir_mez/labels")

For images containing only one weapon:

In [ ]:
dir_origen = "/content/drive/MyDrive/Deteccion_armas_TFG/Datasets/weapons.v2-weapons_blackedges_contrast.yolov5pytorch/test/images/"

# images path
imagenes = glob.glob("/content/drive/MyDrive/Deteccion_armas_TFG/Datasets/weapons.v2-weapons_blackedges_contrast.yolov5pytorch/test/images/*.jpg")

dir_gra = "/content/dir_gra"
dir_med = "/content/dir_med"
dir_peq = "/content/dir_peq"


for img in imagenes:
  image = cv2.imread(img)
  # Image size (PIXELS)
  w = image.shape[0]
  h = image.shape[1]
  image_size = w*h
  real = img.replace("/images/","/labels/",1)
  real = real[:-3] + "txt"
  img_name = re.sub(dir_origen,'',img)
  img_label = img_name[:-3] + "txt"
  print(img_label)
  print(img_name)
  with open(real, 'r+') as archivo:
    lista = archivo.readlines()
    num = len(lista)
    if num == 1:
        linea = lista[0]

        coord1 =[]
        clase = linea[0]
        coord1.append(linea[1:].split())
        coord1 =list(itertools.chain(*coord1))
        x=0
        for i in coord1:
              coord1[x]= float(i)
              x +=1
        # Selecting height and wide values from the coordenates
        w1 = coord1[2] * 416
        h1 = coord1[3] * 416
        # Calculating instance size using those values
        instance_size = w1*h1

        # Calculating percentage of the image ocuppied
        percentage_size = instance_size * 100 / image_size
        print("La instancia ocupa el "+ str(percentage_size) +"% de la imagen")

        if 0 <= percentage_size <= 15:
          shutil.copy(img, "dir_peq/images/"+ img_name)
          shutil.copy(real,"dir_peq/labels/"+ img_label)
          print("Instancia peque")
        elif 16 <= percentage_size <= 35:
          shutil.copy(img, "dir_med/images/"+  img_name)
          shutil.copy(real,"dir_med/labels/"+ img_label)
          print("Instancia mediana")
        else:
          shutil.copy(img, "dir_gra/images/"+  img_name)
          shutil.copy(real,"dir_gra/labels/"+ img_label)
          print("Instancia grande")






For images containing one or more than one weapon:

In [ ]:
dir_origen = "/content/drive/MyDrive/WeaponDetectionYOLOv5/Experimento_2/dataset/test/images/"

imagenes = glob.glob("/content/drive/MyDrive/WeaponDetectionYOLOv5/Experimento_2/dataset/test/images/*.jpg")

dir_gra = "/content/dir_gra"
dir_med = "/content/dir_med"
dir_peq = "/content/dir_peq"
dir_mez = "/content/dir_mez"


for img in imagenes:
  image = cv2.imread(img)

  w = image.shape[0]
  h = image.shape[1]
  image_size = w*h
  real = img.replace("/images/","/labels/",1)
  real = real[:-3] + "txt"
  img_name = re.sub(dir_origen,'',img)
  img_label = img_name[:-3] + "txt"
  print(img_name)
  with open(real, 'r+') as archivo:
    lista = archivo.readlines()
    num = len(lista)
    contador_p = 0
    contador_m = 0
    contador_g = 0
    for linea in lista:
        coord1 =[]
        clase = linea[0]
        coord1.append(linea[1:].split())
        coord1 =list(itertools.chain(*coord1))
        x=0
        for i in coord1:
              coord1[x]= float(i)
              x +=1

        w1 = coord1[2] * 416
        h1 = coord1[3] * 416

        instance_size = w1*h1


        percentage_size = instance_size * 100 / image_size
        print("La instancia ocupa el "+ str(percentage_size) +"% de la imagen")

        # Adding the number of instances of each size
        if 0 <= percentage_size <= 15:
          contador_p = contador_p + 1
        elif 16 <= percentage_size <= 35:
          contador_m = contador_m + 1
        else:
          contador_g = contador_g + 1


    # Assign a specific directory

    if contador_p == num :
          shutil.copy(img, "dir_peq/images/"+ img_name)
          shutil.copy(real,"dir_peq/labels/"+ img_label)
          print("PEQUEÑO")
    elif contador_m == num:
          shutil.copy(img, "dir_med/images/"+  img_name)
          shutil.copy(real,"dir_med/labels/"+ img_label)
          print("MEDIANO")
    elif contador_g == num:
          shutil.copy(img, "dir_gra/images/"+  img_name)
          shutil.copy(real,"dir_gra/labels/"+ img_label)
          print("GRANDE")
    else:
          shutil.copy(img, "dir_mez/images/"+  img_name)
          shutil.copy(real,"dir_mez/labels/"+ img_label)
          print("MEZCLA")




In [ ]:
# Analyze images with size mixing in instances
dir_origen = "/content/dir_mez/images"
imagenes = glob.glob("/content/dir_mez/images/*.jpg")

for img in imagenes:
  image = cv2.imread(img)

  w = image.shape[0]
  h = image.shape[1]
  image_size = w*h
  real = img.replace("/images/","/labels/",1)
  real = real[:-3] + "txt"
  img_name = re.sub(dir_origen,'',img)
  img_label = img_name[:-3] + "txt"
  print(img_name)
  with open(real, 'r+') as archivo:
    lista = archivo.readlines()
    num = len(lista)
    contador_p = 0
    contador_m = 0
    contador_g = 0
    for linea in lista:
        coord1 =[]
        clase = linea[0]
        coord1.append(linea[1:].split())
        coord1 =list(itertools.chain(*coord1))
        x=0
        for i in coord1:
              coord1[x]= float(i)
              x +=1

        w1 = coord1[2] * 416
        h1 = coord1[3] * 416

        instance_size = w1*h1


        percentage_size = instance_size * 100 / image_size
        print("La instancia ocupa el "+ str(percentage_size) +"% de la imagen")


        if 0 <= percentage_size <= 15:
          print("Instancia peque")
          print(linea)
        elif 16 <= percentage_size <= 35:
          print("Instancia mediana")
          print(linea)
        else:
          print("Instancia grande")
          print(linea)

In [ ]:
# Download images by size
from google.colab import files

!zip -r imagenes_medianas.zip /content/dir_med
!zip -r imagenes_pequeñas.zip /content/dir_peq
!zip -r imagenes_grandes.zip /content/dir_gra
!zip -r imagenes_mezcla.zip /content/dir_mez
files.download('imagenes_medianas.zip')
files.download('imagenes_pequeñas.zip')
files.download('imagenes_grandes.zip')
files.download('imagenes_mezcla.zip')

### Modify labels value

>If there is a problem because the dataset classes are changed, this code fragment modifies the class value in the labels.  
Replace  **labels_path** with the path of the directory containing the labels.








In [ ]:
import glob
dir = glob.glob('labels_path/*.txt')

for elem in dir:
 fich1 = open(elem,'r')
 for line in fich1:
      if(line[0] == "0"):
        data = line.replace("0","1",1)
        fich2 = open(elem,'w')
        fich2.write(data)
        fich2.close()
      else:
        data = line.replace("1","0",1)
        fich2 = open(elem,'w')
        fich2.write(data)
        fich2.close()
 fich1.close()





In [ ]:
# Download directory
from google.colab import files

!zip -r labels.zip labels_path
files.download('labels.zip')